In [13]:
!pip install numpy

In [1]:
!pip install dlib

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.6-cp312-cp312-win_amd64.whl size=2924519 sha256=741c639c1bbe4472eda530112ec2bac05621eb72875f523726d19ad90707acde
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\8d\16\b2\d2df2ea5d9430e80343c7b696e9588e5cc98b0d903d434d768
Successfully built dlib


In [3]:
!pip install face-recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)


In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime, date
import sqlite3
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import pandas as pd
import logging
import threading

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
def init_db():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS people 
                 (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL, photo_path TEXT NOT NULL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS attendance 
                 (id INTEGER, name TEXT, timestamp TEXT, FOREIGN KEY(id) REFERENCES people(id))''')
    conn.commit()
    conn.close()
    logging.info("Database initialized.")

In [7]:
def load_known_faces():
    known_faces = {}
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT name, photo_path FROM people")
    rows = c.fetchall()
    if not rows:
        logging.warning("No people found in the database.")
    for name, photo_path in rows:
        try:
            image = face_recognition.load_image_file(photo_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:
                known_faces[name] = [encoding[0]]
                logging.info(f"Loaded face encoding for {name}.")
            else:
                logging.warning(f"No face encoding found in {photo_path} for {name}.")
        except Exception as e:
            logging.error(f"Error loading face for {name}: {e}")
    conn.close()
    return known_faces

In [9]:
class AddPersonGUI:
    def __init__(self, root, status_callback):
        self.root = root
        self.root.title("Add New Person")
        self.root.geometry("640x650")
        self.root.configure(bg="#e6f0fa")
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.status_callback = status_callback

        self.frame = ttk.Frame(self.root, padding="20", style="My.TFrame")
        self.frame.pack(fill='both', expand=True)

        ttk.Label(self.frame, text="Add New Person", font=("Segoe UI", 18, "bold"), foreground="#1e90ff").pack(pady=10)

        ttk.Label(self.frame, text="Enter Name:", font=("Segoe UI", 12)).pack(pady=5)
        self.name_entry = ttk.Entry(self.frame, width=30, font=("Segoe UI", 12))
        self.name_entry.pack(pady=5)

        self.capture_btn = ttk.Button(self.frame, text="📸 Capture Photo", command=self.capture_photo, style="Accent.TButton")
        self.capture_btn.pack(pady=10)
        self.capture_btn.config(state='disabled')

        self.submit_btn = ttk.Button(self.frame, text="✔ Submit", command=self.submit_person, style="Accent.TButton")
        self.submit_btn.pack(pady=10)
        self.submit_btn.config(state='disabled')

        self.video_label = ttk.Label(self.frame, text="Loading camera...", font=("Segoe UI", 12), foreground="gray")
        self.video_label.pack(pady=10, expand=True)

        self.status_var = tk.StringVar(value="Initializing camera...")
        ttk.Label(self.frame, textvariable=self.status_var, font=("Segoe UI", 10), foreground="gray").pack(pady=10)

        self.cap = None
        self.camera_ready = False
        threading.Thread(target=self.init_camera, daemon=True).start()

    def init_camera(self):
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.root.after(0, lambda: messagebox.showerror("Error", "Camera not accessible! Please check your webcam."))
                self.root.after(0, self.on_closing)
                return
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            self.camera_ready = True
            self.root.after(0, self.enable_buttons)
            self.root.after(0, lambda: self.status_var.set("Camera ready"))
            self.root.after(0, lambda: self.status_callback("Camera initialized"))
            self.show_video()
        except Exception as e:
            logging.error(f"Error initializing camera: {e}")
            self.root.after(0, lambda: messagebox.showerror("Error", f"Camera initialization failed: {e}"))
            self.root.after(0, self.on_closing)

    def enable_buttons(self):
        self.capture_btn.config(state='normal')
        self.submit_btn.config(state='normal')
        self.video_label.config(text="")

    def show_video(self):
        if self.camera_ready and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
            self.root.after(10, self.show_video)

    def capture_photo(self):
        if self.camera_ready and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                self.photo = frame
                cv2.imwrite("temp_photo.jpg", frame)
                messagebox.showinfo("Success", "Photo captured!")
                self.status_var.set("Photo captured")
                self.status_callback("Photo captured")
                logging.info("Photo captured successfully.")
            else:
                messagebox.showerror("Error", "Failed to capture photo!")
                self.status_var.set("Capture failed")
                self.status_callback("Capture failed")
                logging.error("Failed to capture photo.")

    def submit_person(self):
        name = self.name_entry.get().strip()
        if not name or not hasattr(self, 'photo'):
            messagebox.showerror("Error", "Please enter a name and capture a photo!")
            self.status_var.set("Error: Missing name or photo")
            self.status_callback("Error: Missing name or photo")
            return

        photo_path = f"dataset/{name}.jpg"
        os.makedirs("dataset", exist_ok=True)
        cv2.imwrite(photo_path, self.photo)

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()
        c.execute("INSERT INTO people (name, photo_path) VALUES (?, ?)", (name, photo_path))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", f"{name} added successfully!")
        self.status_var.set(f"{name} added")
        self.status_callback(f"{name} added to database")
        logging.info(f"{name} added to database.")
        self.on_closing()

    def on_closing(self):
        if hasattr(self, 'cap') and self.cap and self.cap.isOpened():
            self.cap.release()
        self.root.destroy()
        logging.info("Add Person GUI closed.")

In [11]:
def mark_attendance(status_callback):
    logging.info("Starting attendance system...")
    known_faces = load_known_faces()
    if not known_faces:
        messagebox.showwarning("Warning", "No known faces in the database. Please add people first.")
        status_callback("No known faces in database")
        logging.warning("No known faces loaded.")
        return

    loading_win = tk.Toplevel()
    loading_win.title("Loading Attendance")
    loading_win.geometry("300x100")
    loading_win.configure(bg="#e6f0fa")
    ttk.Label(loading_win, text="Initializing camera...", font=("Segoe UI", 12), foreground="gray").pack(pady=20)

    cap = [None]  
    camera_ready = [False]

    def init_camera():
        try:
            cap[0] = cv2.VideoCapture(0)
            if not cap[0].isOpened():
                loading_win.after(0, lambda: messagebox.showerror("Error", "Camera not accessible! Please check your webcam."))
                loading_win.after(0, loading_win.destroy)
                status_callback("Camera not accessible")
                return
            cap[0].set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            cap[0].set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            camera_ready[0] = True
            loading_win.after(0, loading_win.destroy)
            status_callback("Attendance camera ready")
            run_attendance(cap[0])
        except Exception as e:
            logging.error(f"Error initializing camera: {e}")
            loading_win.after(0, lambda: messagebox.showerror("Error", f"Camera initialization failed: {e}"))
            loading_win.after(0, loading_win.destroy)
            status_callback(f"Error: {e}")

    def run_attendance(cap_instance):
        today = date.today().strftime("%Y-%m-%d")
        logging.info(f"Today’s date: {today}")

        try:
            while True:
                ret, frame = cap_instance.read()
                if not ret:
                    messagebox.showerror("Error", "Failed to capture frame from camera!")
                    status_callback("Failed to capture frame")
                    logging.error("Failed to capture frame.")
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    name = "Unknown"
                    message = ""
                    for person_name, encodings in known_faces.items():
                        matches = face_recognition.compare_faces(encodings, face_encoding, tolerance=0.6)
                        if True in matches:
                            name = person_name
                            break

                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(frame, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                    if name != "Unknown":
                        conn = sqlite3.connect('attendance.db')
                        c = conn.cursor()
                        c.execute("SELECT timestamp FROM attendance WHERE name = ? AND timestamp LIKE ?", 
                                  (name, f"{today}%"))
                        existing_record = c.fetchone()

                        if existing_record:
                            message = "Attendance already taken"
                            logging.info(f"{name}: Attendance already taken at {existing_record[0]}")
                            status_callback(f"{name}: Already marked today")
                        else:
                            c.execute("SELECT id FROM people WHERE name = ?", (name,))
                            person_id = c.fetchone()
                            if person_id:
                                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                c.execute("INSERT INTO attendance (id, name, timestamp) VALUES (?, ?, ?)", 
                                          (person_id[0], name, timestamp))
                                conn.commit()
                                message = "Attendance taken"
                                logging.info(f"Attendance marked for {name} at {timestamp}")
                                status_callback(f"{name}: Attendance marked")
                            else:
                                logging.error(f"No ID found for {name} in people table.")

                        conn.close()

                    if message:
                        cv2.putText(frame, message, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                                    (0, 255, 0) if message == "Attendance taken" else (0, 0, 255), 2)

                cv2.imshow('Attendance System', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    logging.info("Attendance system stopped by user.")
                    status_callback("Attendance system stopped")
                    break
        except Exception as e:
            logging.error(f"Error in attendance system: {e}")
            status_callback(f"Error: {e}")
        finally:
            if cap_instance.isOpened():
                cap_instance.release()
            cv2.destroyAllWindows()
            logging.info("Attendance system resources cleaned up.")

    threading.Thread(target=init_camera, daemon=True).start()

In [13]:
def view_attendance(status_callback):
    conn = sqlite3.connect('attendance.db')
    df = pd.read_sql_query("SELECT * FROM attendance", conn)
    conn.close()

    if df.empty:
        messagebox.showinfo("Attendance", "No attendance records found!")
        status_callback("No attendance records found")
        return

    top = tk.Toplevel()
    top.title("Attendance Records")
    top.geometry("500x400")
    top.configure(bg="#e6f0fa")
    top.protocol("WM_DELETE_WINDOW", lambda: top.destroy())

    frame = ttk.Frame(top, padding="20", style="My.TFrame")
    frame.pack(fill='both', expand=True)

    ttk.Label(frame, text="Attendance Records", font=("Segoe UI", 18, "bold"), foreground="#1e90ff").pack(pady=10)

    text = tk.Text(frame, height=15, width=60, font=("Segoe UI", 10))
    text.pack(expand=True, fill='both')
    text.insert(tk.END, df.to_string(index=False))
    text.config(state='disabled')

    status_callback("Viewing attendance records")

In [ ]:
def main():
    init_db()
    root = tk.Tk()
    root.title("Face Recognition Attendance System")
    root.geometry("400x350")
    root.configure(bg="#e6f0fa")
    root.protocol("WM_DELETE_WINDOW", lambda: on_closing(root))
    root.resizable(True, True)  # Allow resizing

    # Apply a modern theme
    style = ttk.Style()
    style.theme_use('clam')
    style.configure("My.TFrame", background="#e6f0fa")
    style.configure("TButton", font=("Segoe UI", 12), padding=10)
    style.configure("Accent.TButton", background="#1e90ff", foreground="white")
    style.map("Accent.TButton", background=[('active', '#4169e1')])

    # Scrollable canvas
    canvas = tk.Canvas(root, bg="#e6f0fa")
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas, style="My.TFrame")

    scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    def on_mouse_wheel(event):
        canvas.yview_scroll(-1 * (event.delta // 120), "units")

    root.bind_all("<MouseWheel>", on_mouse_wheel)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")
    
    ttk.Label(scrollable_frame, text="Attendance System", font=("Segoe UI", 20, "bold"), foreground="#1e90ff").pack(pady=20)

    ttk.Button(scrollable_frame, text="➕ Add New Person", 
               command=lambda: AddPersonGUI(tk.Toplevel(root), lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="▶ Start Attendance", 
               command=lambda: mark_attendance(lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="📋 View Attendance", 
               command=lambda: view_attendance(lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="✖ Exit", 
               command=lambda: on_closing(root), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)

   
    status_var = tk.StringVar(value="System Ready")
    ttk.Label(scrollable_frame, textvariable=status_var, font=("Segoe UI", 10), foreground="gray").pack(pady=10)

    root.mainloop()

def on_closing(root):
    logging.info("Closing application...")
    cv2.destroyAllWindows()
    root.quit()
    root.destroy()
    logging.info("Application closed successfully.")

if __name__ == "__main__":
    main()

2025-07-02 00:38:29,711 - INFO - Database initialized.
2025-07-02 00:39:04,665 - INFO - Photo captured successfully.
2025-07-02 00:39:08,273 - INFO - Pranay added to database.
2025-07-02 00:39:08,557 - INFO - Add Person GUI closed.
2025-07-02 00:39:10,463 - INFO - Starting attendance system...
2025-07-02 00:39:11,126 - INFO - Loaded face encoding for Ritik.
2025-07-02 00:39:11,763 - INFO - Loaded face encoding for John.
2025-07-02 00:39:12,354 - INFO - Loaded face encoding for Ritik.
2025-07-02 00:39:12,942 - INFO - Loaded face encoding for Ritik.
2025-07-02 00:39:13,526 - INFO - Loaded face encoding for Ritik.
2025-07-02 00:39:14,121 - INFO - Loaded face encoding for Santosh.
2025-07-02 00:39:14,736 - INFO - Loaded face encoding for vaibhav.
2025-07-02 00:39:15,321 - INFO - Loaded face encoding for rudra sir.
2025-07-02 00:39:15,938 - INFO - Loaded face encoding for Pranay.
2025-07-02 00:39:16,638 - INFO - Today’s date: 2025-07-02
2025-07-02 00:39:17,723 - INFO - Attendance marked for